In [ ]:
# !pip install anthropic
# !pip install python-dotenv


In [7]:
import csv
import random
import anthropic
import os
import sys
import time
import llm_interfaces
import prompts

In [8]:
interface = 'anthropic'

In [9]:

def send_prompt(clue: str, resolution_index: int):
    iterations = 3
    mapped_messages = []

    for i in range(iterations):
        new_user_message = llm_interfaces.mount_user_prompt(clue, resolution_index, i)
        mapped_messages.append(new_user_message)
        
        new_response = llm_interfaces.call_model(mapped_messages)
        
        if new_response is False or new_response is None:
            break
            
        new_assistant_message = llm_interfaces.mount_assistant_prompt(new_response)
        mapped_messages.append(new_assistant_message)
        
    mapped_messages.insert(0, "\n\n\n============ USER ============\n\n\n")
    mapped_messages.insert(2, "\n\n\n========== ASSISTANT ==========\n\n\n")
    mapped_messages.insert(4, "\n\n\n============ USER ============\n\n\n")
    mapped_messages.insert(6, "\n\n\n========== ASSISTANT ==========\n\n\n")
    mapped_messages.insert(8, "\n\n\n============ USER ============\n\n\n")
    mapped_messages.insert(10, "\n\n\n========== ASSISTANT ==========\n\n\n")
    
    return mapped_messages
    
def save_result(mapped_messages, resolution_index: int):
    result_file = prompts.resolution_methods[resolution_index]["result_file"]
    os.remove(result_file)
    
    f = open(result_file, "x")
    f.write("".join([prompts.get_response_text(message) for message in mapped_messages]))
    
    f.close()
    
    return f"{result_file} done!"

In [ ]:
clue = """Tentei subir até o novo comunista que veio de fora, mas cansei bem no começo"""

result = None
for i in range(len(resolution_methods)):
    result = send_prompt(clue, i)
    save_result(result, i)
    print(i, "done!")
    